## Importing things, JSON reader function

In [8]:
import pandas as pd
import numpy as np
import re, string
import json
import os
import ast

Reading business.json
Creating business_sample.json
Getting business_ids
Reading review.json
Creating review_sample.json line by line
Getting user_ids


"\n# USER.JSON\nuser_data = []\nwith open(os.path.dirname(__file__) + user_file) as f:\n    for line in f:\n        newline = ast.literal_eval(line)  # read the line (str) as a dict\n        if newline['user_id'] in user_ids:\n            user_data.append(json.loads(line))\nuser_df = pd.DataFrame.from_dict(user_data)\nprint 'Creating user_sample.json'\nwith open('user_sample.json', 'w') as f:\n    user_df.to_json(f, orient='records', lines=True)\n\n# TIP.JSON\ntip_data = []\nprint 'Reading tip.json'\nwith open(os.path.dirname(__file__) + tip_file) as f:\n    for line in f:\n        newline = ast.literal_eval(line)\n        if newline['business_id'] in business_ids:\n            tip_data.append(json.loads(line))\ntip_df = pd.DataFrame.from_dict(tip_data)\nprint 'Creating tip_sample.json'\nwith open('tip_sample.json', 'w') as f:\n    tip_df.to_json(f, orient='records', lines=True)\n\n# CHECKIN.JSON\ncheckin_data = []\nprint 'Reading checkin.json'\nwith open(os.path.dirname(__file__) + ch

In [ ]:
# Code copied from https://www.kaggle.com/vksbhandary/exploring-yelp-reviews-dataset
def init_ds(json):
    ds= {}
    keys = json.keys()
    for k in keys:
        ds[k]= []
    return ds, keys

def read_json(file):
    dataset = {}
    keys = []
    with open(file) as file_lines:
        for count, line in enumerate(file_lines):
            data = json.loads(line.strip())
            if count ==0:
                dataset, keys = init_ds(data)
            for k in keys:
                dataset[k].append(data[k])
                
        return pd.DataFrame(dataset)

## Getting Yelp Business Data

### Overall

In [ ]:
businesses = read_json('yelp_academic_dataset_business.json')
print(businesses.columns)
print(businesses.shape)

# Filter for places that are open, is_open = 1
businesses = businesses[businesses.is_open == 1]
# Filter out places that don't have any category
businesses = businesses.dropna(subset = ['categories'])
print(businesses.shape)

### Filtering for PGH/LV

In [ ]:
# Filter for Pittsburgh
# Typos to consider: Pittsburg, Pittsburch, Pitsburgh, Pittsbuirgh, PITTSBURGH AP
pitt_string = (businesses.city.str.strip().str.lower() == 'pittsburgh') | \
              (businesses.city.str.strip().str.lower() == 'pittsburg') | \
              (businesses.city.str.strip().str.lower() == 'pittsburch') | \
              (businesses.city.str.strip().str.lower() == 'pitsburgh') | \
              (businesses.city.str.strip().str.lower() == 'pittsbuirgh') | \
              (businesses.city.str.strip().str.lower() == 'pittsburgh ap') 
pa_string = businesses.state == 'PA'
pittsburgh = businesses[(pitt_string) & (pa_string)]

# Convert all to Pittsburgh to make it cleaner
pittsburgh.loc[:, "city"] = np.repeat('Pittsburgh', len(pittsburgh))
pittsburgh.city.value_counts()

In [ ]:
# Filter for Las Vegas
# Typos to consider: Las Vegas Ap; Las Vegas East; Las Vegas Nv; Las Vegas Nevada; Las Vegas, Nevada; Las Vegas, NV; Las Vegass; Las Vegas,
lv_string = (businesses.city.str.strip().str.lower() == 'las vegas') | \
            (businesses.city.str.strip().str.lower() == 'las vegas ap') | \
            (businesses.city.str.strip().str.lower() == 'las vegas east') | \
            (businesses.city.str.strip().str.lower() == 'las vegas nv') | \
            (businesses.city.str.strip().str.lower() == 'las vegas nevada') | \
            (businesses.city.str.strip().str.lower() == 'las vegas,') | \
            (businesses.city.str.strip().str.lower() == 'las vegas, nevada') | \
            (businesses.city.str.strip().str.lower() == 'las vegas, nv') | \
            (businesses.city.str.strip().str.lower() == 'las vegass')
nv_string = businesses.state == 'NV'
las_vegas = businesses[(lv_string) & (nv_string)]

# Convert all to Pittsburgh to make it cleaner
las_vegas.city = np.repeat('Las Vegas', len(las_vegas))

In [ ]:
with open('pittsburgh_sample.json', 'w') as f:
    pittsburgh.to_json(f, orient='records', lines=True)
    
with open('las_vegas_sample.json', 'w') as f:
    las_vegas.to_json(f, orient='records', lines=True)

### Finding top 100 + their business IDs

In [ ]:
top_100_pgh = pittsburgh.sort_values(by = 'review_count', ascending = False)[:100]
top_100_lv = las_vegas.sort_values(by = 'review_count', ascending = False)[:100]

In [ ]:
pitt_100_ids = pd.Series(top_100_pgh['business_id'].index.values, index=top_100_pgh['business_id']).to_dict()
lv_100_ids = pd.Series(top_100_lv['business_id'].index.values, index=top_100_lv['business_id']).to_dict()

## Reviews data - top 100 busineses

In [ ]:
review_data = []
print('Reading review.json')
with open('yelp_academic_dataset_review.json') as f:
    for line in f:
        newline = ast.literal_eval(line)  # read the line (str) as a dict
        if newline['business_id'] in pitt_100_ids:
            review_data.append(json.loads(line))
print('converting to data frame')
review_df = pd.DataFrame.from_dict(review_data)
# get list of business id's from business.json and only keep reviews of those businesses
# would normally write it with to_json but it's too big to fit in memory, so write it line by line
print('Creating review_sample.json line by line')
with open('pitt_reviews_sample.json', 'w') as f:
    review_df.to_json(f, orient='records', lines=True)
    
print('done!')

In [ ]:
review_data = []
print('Reading review.json')
with open('yelp_academic_dataset_review.json') as f:
    for line in f:
        newline = ast.literal_eval(line)  # read the line (str) as a dict
        if newline['business_id'] in lv_100_ids:
            review_data.append(json.loads(line))
print('converting to data frame')
review_df = pd.DataFrame.from_dict(review_data)
# get list of business id's from business.json and only keep reviews of those businesses
# would normally write it with to_json but it's too big to fit in memory, so write it line by line
print('Creating review_sample.json line by line')
with open('lv_reviews_sample.json', 'w') as f:
    review_df.to_json(f, orient='records', lines=True)

## Tips data - top 100 businesses

In [ ]:
tip_data = []
print('Reading tip.json')
with open('yelp_academic_dataset_tip.json') as f:
    for line in f:
        newline = ast.literal_eval(line)
        if newline['business_id'] in pitt_100_ids:
            tip_data.append(json.loads(line))
print('Converting to DF')
tip_df = pd.DataFrame.from_dict(tip_data)
print('Creating json file')
with open('tip_pittsburgh.json', 'w') as f:
    tip_df.to_json(f, orient='records', lines=True)

In [ ]:
tip_data = []
print('Reading tip.json')
with open('yelp_academic_dataset_tip.json') as f:
    for line in f:
        newline = ast.literal_eval(line)
        if newline['business_id'] in lv_100_ids:
            tip_data.append(json.loads(line))
print('Converting to DF')
tip_df = pd.DataFrame.from_dict(tip_data)
print('Creating json file')
with open('tip_lv.json', 'w') as f:
    tip_df.to_json(f, orient='records', lines=True)

## Check-ins data - top 100 businesses

In [ ]:
checkin_data = []
print('Reading checkin.json')
with open('yelp_academic_dataset_checkin.json') as f:
    for line in f:
        newline = ast.literal_eval(line)
        if newline['business_id'] in pitt_100_ids:
            checkin_data.append(json.loads(line))
checkin_df = pd.DataFrame.from_dict(checkin_data)
print('Creating checkin_sample.json')
with open('pitt_checkin.json', 'w') as f:
    checkin_df.to_json(f, orient='records', lines=True)
print('Done!')

In [ ]:
checkin_data = []
print('Reading checkin.json')
with open('yelp_academic_dataset_checkin.json') as f:
    for line in f:
        newline = ast.literal_eval(line)
        if newline['business_id'] in lv_100_ids:
            checkin_data.append(json.loads(line))
checkin_df = pd.DataFrame.from_dict(checkin_data)
print('Creating checkin_sample.json')
with open('lv_checkin.json', 'w') as f:
    checkin_df.to_json(f, orient='records', lines=True)
print('Done!')